https://medium.com/@datadrifters

install required libraries

In [ ]:
%pip install -q accelerate xformers
%pip install -q git+https://github.com/huggingface/transformers.git@main
%pip install -q optimum auto-gptq langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.0/167.0 MB 7.0 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
import torch, transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
dtype = torch.bfloat16 if torch.cuda.get_device_capability()[0] == 8 else torch.float16
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
#MODEL_ID = 'tiiuae/falcon-7b-instruct'
#model_id = "meta-llama/Llama-2-13b-chat-hf"
model_id = "TheBloke/Llama-2-7b-Chat-GPTQ"

Loading Model

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype = dtype,
    trust_remote_code=True
    )

print(f"Model device: {model.device}")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id,  padding_side="right", truncation=True)
tokenizer.pad_token = tokenizer.eos_token

Installing langchain & relevant packages

In [ ]:
from transformers import AutoTokenizer, pipeline

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    top_p=0.95,
    repetition_penalty=1.15,
    #prompt_template
)

from langchain.llms import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=pipe)

In [1]:
!pip3 install -qq langchain chromadb arxiv pymupdf
!pip3 install -qq sentence_transformers pypdf unstructured

# Importing required libraries

In [2]:
# Imports
from chromadb.config import Settings
from urllib.error import HTTPError
from dataclasses import replace
#from dotenv import load_dotenv
from tqdm import tqdm
import numpy as np
#import tiktoken # OpenAI's open-source tokenizer
import chromadb
#import logging
import random # to sample multiple elements from a list
import arxiv
import time
import os # operating system dependent functionality, to walk through directories and files

from langchain.text_splitter import RecursiveCharacterTextSplitter # recursively tries to split by different characters to find one that works
from langchain.document_loaders import PyPDFDirectoryLoader # loads pdfs from a given directory
from langchain.chains import ConversationalRetrievalChain # looks up relevant documents from the retriever per history and question.
from langchain.text_splitter import CharacterTextSplitter # splits the content
from langchain.embeddings import HuggingFaceBgeEmbeddings # wrapper for HuggingFaceBgeEmbeddings models
from langchain import PromptTemplate, LLMChain
from langchain.document_loaders import ArxivLoader # loads paper for a given id from Arxiv
from langchain.document_loaders import PyPDFLoader # loads a given pdf
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader # loads a given text
from langchain.retrievers import ArxivRetriever # loads relevant papers for a given paper id from Arxiv
from chromadb.utils import embedding_functions # loads Chroma's embedding functions from OpenAI, HuggingFace, SentenceTransformer and others
from langchain.vectorstores import Chroma # wrapper around ChromaDB embeddings platform
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import HuggingFaceHub # wrapper around HuggingFaceHub models

!pip install -qq html2text
from langchain.document_loaders import WebBaseLoader
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import BSHTMLLoader


import warnings
warnings.filterwarnings("ignore")

creating the HuggingFacePipeline

# Downloading HuggingFace BGE Embeddings

In [4]:
model_name = "BAAI/bge-base-en"
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embedding_function = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cpu'},
    encode_kwargs=encode_kwargs
)

# Working with ChromaDB to store embeddings

In [5]:
!pip install -qq requests beautifulsoup4 tqdm faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 69.7 MB/s eta 0:00:00


code 2

In [8]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import json
from pathlib import Path
import pickle
import os

main_web_url='https://docs.ray.io/en/latest/rllib/index.html'
full_url='https://www.neduet.edu.pk/'
domain = "neduet.edu.pk"
#url = 'https://docs.ray.io/en/latest/rllib/index.html'

dicnry_links={}
#main_web_url='https://www.udst.edu.qa'
#url = 'https://www.udst.edu.qa'
dicnry_links[url]=0
counter=0

destination_dir="web_data"
!rm -rf 'web_data'
dest_f=main_web_url.split("://")[-1]
print(dest_f)
destination_dir=destination_dir+'/'+dest_f.replace(".","_")
print(destination_dir)

if not os.path.isdir(destination_dir):
    os.makedirs(destination_dir, mode=511)

# now go through the link and download pages as html or pdf

docs.ray.io/en/latest/rllib/index.html
web_data/docs_ray_io/en/latest/rllib/index_html


In [21]:
from html import parser
import requests
import re
import urllib.request
from bs4 import BeautifulSoup
from collections import deque
from html.parser import HTMLParser
from urllib.parse import urlparse
import os

# Regex pattern to match a URL
HTTP_URL_PATTERN = r'^http[s]*://.+'

# Define root domain to crawl
#domain = "openai.com"
#full_url = "https://openai.com/"

# Create a class to parse the HTML and get the hyperlinks
class HyperlinkParser(HTMLParser):
    def __init__(self):
        super().__init__()
        # Create a list to store the hyperlinks
        self.hyperlinks = []

    # Override the HTMLParser's handle_starttag method to get the hyperlinks
    def handle_starttag(self, tag, attrs):
        attrs = dict(attrs)

        # If the tag is an anchor tag and it has an href attribute, add the href attribute to the list of hyperlinks
        if tag == "a" and "href" in attrs:
            self.hyperlinks.append(attrs["href"])

# Function to get the hyperlinks from a URL
def get_hyperlinks(url):

    # Try to open the URL and read the HTML
    try:
        # Open the URL and read the HTML
        with urllib.request.urlopen(url) as response:

            # If the response is not HTML, return an empty list
            if not response.info().get('Content-Type').startswith("text/html"):
                return []

            # Decode the HTML
            html = response.read().decode('utf-8')
    except Exception as e:
        print(e)
        return []

    # Create the HTML Parser and then Parse the HTML to get hyperlinks
    parser = HyperlinkParser()
    parser.feed(html)

    return parser.hyperlinks

# Function to get the hyperlinks from a URL that are within the same domain
def get_domain_hyperlinks(local_domain, url):
    clean_links = []
    for link in set(get_hyperlinks(url)):
        clean_link = None

        # If the link is a URL, check if it is within the same domain
        if re.search(HTTP_URL_PATTERN, link):
            # Parse the URL and check if the domain is the same
            url_obj = urlparse(link)
            if url_obj.netloc == local_domain:
                clean_link = link

        # If the link is not a URL, check if it is a relative link
        else:
            if link.startswith("/"):
                link = link[1:]
            elif link.startswith("#") or link.startswith("mailto:"):
                continue
            clean_link = "https://" + local_domain + "/" + link

        if clean_link is not None:
            if clean_link.endswith("/"):
                clean_link = clean_link[:-1]
            clean_links.append(clean_link)

    # Return the list of hyperlinks that are within the same domain
    return list(set(clean_links))


def crawl(url):
    # Parse the URL and get the domain
    local_domain = urlparse(url).netloc

    # Create a queue to store the URLs to crawl
    queue = deque([url])

    # Create a set to store the URLs that have already been seen (no duplicates)
    seen = set([url])

    # Create a directory to store the text files
    if not os.path.exists("text/"):
            os.mkdir("text/")

    if not os.path.exists("text/"+local_domain+"/"):
            os.mkdir("text/" + local_domain + "/")

    # Create a directory to store the csv files
    if not os.path.exists("processed"):
            os.mkdir("processed")

    # While the queue is not empty, continue crawling
    while queue:

        # Get the next URL from the queue
        url = queue.pop()
        print(url) # for debugging and to see the progress

        # Save text from the url to a <url>.txt file
        with open('text/'+local_domain+'/'+url[8:].replace("/", "_") + ".txt", "w") as f:

            # Get the text from the URL using BeautifulSoup
            request = requests.get(url, verify=False)
            soup = BeautifulSoup(request, parser="html.parser", verify_ssl=False)

            # Get the text but remove the tags
            text = soup.get_text()

            # If the crawler gets to a page that requires JavaScript, it will stop the crawl
            if ("You need to enable JavaScript to run this app." in text):
                print("Unable to parse page " + url + " due to JavaScript being required")

            # Otherwise, write the text to the file in the text directory
            f.write(text)

        # Get the hyperlinks from the URL and add them to the queue
        for link in get_domain_hyperlinks(local_domain, url):
            if link not in seen:
                queue.append(link)
                seen.add(link)

crawl(full_url)

https://www.neduet.edu.pk/


TypeError: ignored

In [ ]:
#import urllib2
import urllib.request as urllib2
from urllib.parse import urljoin

from bs4 import *
#from urlparse import urljoin

def crawl(pages, depth=None):
    indexed_url = [] # a list for the main and sub-HTML websites in the main website
    for i in range(depth):
        for page in pages:
            if page not in indexed_url:
                indexed_url.append(page)
                try:
                    c = urllib2.urlopen(page)
                except:
                    print("Could not open %s" % page)
                    continue
                soup = BeautifulSoup(c.read())
                data = [ele.text for ele in soup.find_all(text = True) if ele.text.strip() != '']
                print(data)

                links = soup('a') #finding all the sub_links
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = urljoin(page, link['href'])
                        if url.find("'") != -1:
                            continue
                        url = url.split('#')[0]
                        if url[0:4] == 'http':
                            indexed_url.append(url)
        pages = indexed_url
    return indexed_url


pagelist=[url]
urls = crawl(pagelist, depth=2)
print(urls)

for url in urls:
    try:
        print(url)
        if str(url).endswith(".pdf") or str(url).endswith(".PDF"):
            url=str(url).lower()
            file_name=url.split(".pdf")[-2]
            file_name=file_name.split("/")[-1]
            file_name=file_name+".pdf"
            filename = Path(destination_dir+"/"+file_name)
            response = requests.get(url)
            filename.write_bytes(response.content)

            print("pdf file name",file_name)
            continue

        response = requests.get(url)
        if not '.' in str(url):
            filename = url+'.html'

        file_name=url

        file_name=file_name.replace("/","_")
        print(file_name)
        with open(destination_dir+"/"+file_name,"w") as f:
            f.write(response.text)
    except:
        print("issue")
        pass

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import os

DATA_PATH="data/"
DB_FAISS_PATH = "vectorstores/db_faiss/"

links_path = "/content/web_data/docs_ray_io/en/latest/rllib/index_html"

def create_vector_db(links_path):
    documents=[]
    processed_htmls=0
    processed_pdfs=0
    links_list = [l for l in os.listdir(links_path)]

    for f in links_list: #os.listdir(links_path):#web_data"):
        try:
            if f.endswith(".pdf"):
                pdf_path = os.path.join(links_path,f)#'./web_data/' + f
                loader = PyPDFLoader(pdf_path)
                documents.extend(loader.load())
                processed_pdfs+=1
            elif f.endswith(".html"):
                html_path = os.path.join(links_path,f)#'./web_data/' + f
                #loader = BSHTMLLoader(html_path)
                loader = WebBaseLoader(html_path)#, verify_ssl=False)
                loader.load()
                documents.extend(loader.load())
                processed_htmls+=1
        except:
            print("issue with ",f)
            pass

    print("Processed",processed_htmls,"html files and ",processed_pdfs,"pdf files")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts=text_splitter.split_documents(documents)

    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
        model_kwargs={'device':'cpu'})

    db=FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)
    print(texts)

if __name__=="__main__":
    create_vector_db(links_path)

In [ ]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA

DB_FAISS_PATH = "vectorstores/db_faiss/"

custom_prompt_template='''Use the following pieces of information to answer the users question.
If you don't know the answer, please just say that you don't know the answer. Don't make up an answer.

Context:{context}
question:{question}

Only returns the helpful anser below and nothing else.
Helpful answer
'''

def set_custom_prompt():
    '''
    Prompt template for QA retrieval for each vector store
    '''
    prompt =PromptTemplate(template=custom_prompt_template, input_variables=['context','question'])
    return prompt

def retrieval_qa_chain(llm,prompt,db):
    qa_chain=RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={'k':4}),
    return_source_documents=True,
    chain_type_kwargs={'prompt':prompt}
    )
    return qa_chain

def qa_bot():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
    model_kwargs={'device':'cpu'})
    db = FAISS.load_local(DB_FAISS_PATH,embeddings)
    qa_prompt=set_custom_prompt()
    qa = retrieval_qa_chain(llm,qa_prompt,db)
    return qa

def final_result(query):
    qa_result=qa_bot()
    response=qa_result({'query':query})
    return response

res=final_result('Hi')
answer=res["result"]
sources=res["source_documents"]

if sources:
    answer+=f"\nSources: "+str(str(sources))
else:
    answer+=f"\nNo Sources found"

print(answer)

In [ ]:
res=final_result('I want to know more about the website which is been retrieved for qa')
answer=res["result"]
sources=res["source_documents"]

if sources:
    answer+=f"\nSources: "+str(str(sources))
else:
    answer+=f"\nNo Sources found"

print(answer)

In [ ]:
web_links = [
    'https://docs.ray.io/'
    ]

loader = WebBaseLoader(web_links, verify_ssl=False)
documents = loader.load()

html2text = Html2TextTransformer()
texts = html2text.transform_documents(documents)

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
web_chunks = text_splitter.split_documents(texts)

In [ ]:
web_chunks

In [ ]:
persist_directory="./chromadb/"

vectordb = Chroma.from_documents(
    documents=web_chunks, # text data that you want to embed and store
    embedding=embedding_function, # used to convert the documents into embeddings
    persist_directory=persist_directory, # this tells Chroma where to store its data
    collection_name="RLIB_experiment" #  gives a name to the collection of embeddings, which will be helpful for retrieving specific groups of embeddings later.
)

vectordb.persist() # will make the database save any changes to the disk

# Retrieval QA with LangChain and Chroma

In case you run this code block second time after ChromaDB is created, you can use below line to create vectordb from ChromaDB. This will save time.

In [ ]:
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

creating the QA chain with retriever to answer the questions

In [ ]:
RetrievalQA.from_chain_type.__doc__

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

retrieval_qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

We are ready to ask questions!

In [ ]:
query = "What is RLIB site all about?"
llm_response = retrieval_qa_chain(query)

In [ ]:
llm_response['result'].split('\n')

You can also use retrieval QA chain with prompt templates, here's how you would do it for the same example as above:

In [ ]:
template = """
{summaries}
{question}
"""

retrieval_qa_chain_s = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["summaries", "question"],
        ),
    },
)

In [ ]:
query = "What are RLib Key Concepts?"
llm_response = retrieval_qa_chain_s(query)

In [ ]:
llm_response

We can also see the source and pages to generate the answer

In [ ]:
[source.metadata for source in llm_response["source_documents"]]

## *RLIB* Project

In [ ]:
web_chunks

In [ ]:
#!pip install -q tj_preproc
#from tj_text.tj_preproc import text_preprocessing_ds, text_preprocessing

Loading Documents from website

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!rm -rf pdfs
!wget -r -P pdfs -A pdf "https://www.neduet.edu.pk/content/news" --no-check-certificate -q

In [ ]:
!gdown --fuzzy "https://drive.google.com/file/d/1dpYsXrgckTTSD0agwCe-FX6MIREF-wW3/view?usp=sharing"

In [ ]:
!sudo apt install poppler-utils # required for image ocr
!pip install -q pytesseract # required for image ocr
!pip install -q pdf2image
!pip install -q pymupdf # required for fitz ocr
!pip install -q tesseract

from IPython.display import HTML
import pdf2image
#import pytesseract#, tesseract
from pdf2image import convert_from_path
import tesseract

In [ ]:
import os
import shutil
from pathlib import Path

def copy_pdf_files(source_folder: str, destination_folder: str):
    # Create the destination folder if it doesn't exist
    Path(destination_folder).mkdir(parents=True, exist_ok=True)

    # Get a list of all PDF files in the source folder and its subfolders
    pdf_files = []
    for root, dirs, files in os.walk(source_folder):
        for file in files:
            if file.endswith(".pdf"):
                pdf_files.append(os.path.join(root, file))

    # Copy the PDF files to the destination folder  (done for diagnostic purpose)
    for pdf_file in pdf_files:
        shutil.copy2(pdf_file, destination_folder)

    # Sort the PDF files by creation date
    pdf_files.sort(key=lambda x: os.path.getmtime(x), reverse=True)
    return pdf_files

#!ls -all pdf_folder

In [ ]:
!rm -rf pdf_folder  # removing folder if exist
pdf_files = copy_pdf_files('pdfs', 'pdf_folder')   # copying files
pdf_files.append('/content/CV_Najeed_Updated_May 2023.pdf')

In [ ]:
pdf_files

### Converting pdf to text (scanned pdf)

In [ ]:
%%script true
!sudo apt install tesseract-ocr  # required for 1st algo
!pip install -q pdf2image

import pdf2image
import pytesseract#, tesseract
from pdf2image import convert_from_path
import glob
'''
pdfs = glob.glob(r"yourPath\*.pdf")
for pdf_path in pdfs:
    pages = convert_from_path(pdf_path, 500)

    for pageNum,imgBlob in enumerate(pages):
        text = pytesseract.image_to_string(imgBlob, lang='eng')

        with open(f'{pdf_path[:-4]}_page{pageNum}.txt', 'w') as the_file:
            the_file.write(text)
'''

#### Reading text pdf file

In [ ]:
%%script true
from PyPDF2 import PdfReader
def process_pdf(file_path):
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

#### Reading as scanned pdf

In [ ]:
from pdf2image import convert_from_path
import pytesseract

def process2_pdf(file_path):  #for scanned images

    pages = convert_from_path(file_path, 500)
    text2=''
    for pageNum,imgBlob in enumerate(pages):
        text2 += pytesseract.image_to_string(imgBlob, lang='eng')

    return text2

#### Reading as text pdf

In [ ]:
import fitz

def process3_pdf(file_path):

    text3 = ""
    doc = fitz.open(file_path)
    for page in doc:
        text3 += page.get_text()

    return text3

In [ ]:
#process2_pdf(pdf_files[3])

In [ ]:
loader = DirectoryLoader('./pdf_folder/', glob="./*.pdf", loader_cls=PyPDFLoader)
pdfs = loader.load()

In [ ]:
len(pdfs)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 50
)

pdfs_chunks = text_splitter.split_documents(pdfs)

In [ ]:
%%script true
chunks = []
for f in pdf_files:
    text = process3_pdf(f)
    if text =='': text = process2_pdf(f)

    chunk = text_splitter.split_text(text)
    print(chunk)
    chunks.extend('\n\n')
    chunks.extend(chunk)
print(chunks)

In [ ]:
'''doc_chunks = []
for chunk in [pdfs_chunks, web_chunks]:
    for doc in chunk:
        doc_chunks.append(doc)
'''
doc_chunks = pdfs_chunks

In [ ]:
len(web_chunks), len(pdfs_chunks), len(doc_chunks)

In [ ]:
#for i in doc_chunks:
#    print(type(i))
    #print(i)

In [ ]:
!rm -rf chroma_db
persist_directory="./chromadb/"

vectordb = Chroma.from_documents(
    documents=doc_chunks, # text data that you want to embed and store
    #documents=paper_chunks, # text data that you want to embed and store
    embedding=embedding_function, # used to convert the documents into embeddings
    persist_directory=persist_directory, # this tells Chroma where to store its data
    collection_name="NED_news_and_events" #  gives a name to the collection of embeddings, which will be helpful for retrieving specific groups of embeddings later.
)

vectordb.persist() # will make the database save any changes to the disk

# Retrieval QA with LangChain and Chroma

In case you run this code block second time after ChromaDB is created, you can use below line to create vectordb from ChromaDB. This will save time.

In [ ]:
#vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding_function)

creating the QA chain with retriever to answer the questions

In [ ]:
RetrievalQA.from_chain_type.__doc__

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={"k": 5})

retrieval_qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

We are ready to ask questions!

In [ ]:
query = "Is there any notification from NED University saying holiday in 1, September 2023?"
llm_response = retrieval_qa_chain(query)

In [ ]:
llm_response['result'].split('\n')

You can also use retrieval QA chain with prompt templates, here's how you would do it for the same example as above:

In [ ]:
template = """
You are an AI Assistant who responsd queries at NED University helpdesk. You are polite, helpful and gives as much
infromation as you can. If you don't know anything, you politely say I am sorry please consult NED website or helpline.
{summaries}
Human:{question}
Assistant:
"""

retrieval_qa_chain_s = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PromptTemplate(
            template=template,
            input_variables=["summaries", "question"],
        ),
    },
)

In [ ]:
query = "Tell me about university in 4 lines?"
llm_response = retrieval_qa_chain_s(query)

In [ ]:
llm_response['answer']

We can also see the source and pages to generate the answer

In [ ]:
[source.metadata for source in llm_response["source_documents"]]

In [ ]:
history=''
Q=[
    "Q1) what are the requirements for admission on migration basis in NED university?",
    "Q2) Will there be a holiday for NED on Friday, 1st of September?",
    "Q3) What data should be checked for correction for the students who have passed NED’s Pre-Admission Entry Test?",
    "Q4) List the academic programs offered by NED for undergraduate students?",
    "Q5) Provide details regarding undergraduate admissions in NED for the 2023 academic year.",
    "Q6) List the minimum requirements to apply for admission in NED.",
    "Q7) What are the requirements of admission in the category of differently able candidate?",
    "Q8) Outline the fee structure for undergraduate and postgraduate degrees in NED.",
    "Q9) What is the process of admission for foreign students in NED?",
    "Q10) Who is the current vice chancellor and pro vice chancellor of NED University?"
    ]

In [ ]:
for q in Q:
    history=''
    llm_response = retrieval_qa_chain(q)
    a = llm_response['result']
    display(q)
    display("A:",a)
    print('\n')